In [1]:
pip install kagglehub

In [2]:
import os
import kagglehub
import numpy as np
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import confusion_matrix, roc_curve

# Baixar o dataset
path = kagglehub.dataset_download("arifmia/animal")

# Caminho para o diretório raiz do dataset
dataset_path = path + "/Dataset/train/"
output_path = "./data/"
print("Dataset baixado:", path)



for class_name in ["cat", "dog"]:
    os.makedirs(os.path.join(output_path, class_name), exist_ok=True)
    src_dir = os.path.join(dataset_path, class_name)
    dest_dir = os.path.join(output_path, class_name)
    for img_file in os.listdir(src_dir)[:200]:  # Pegando apenas 200 imagens
        shutil.copy(os.path.join(src_dir, img_file), dest_dir)

# Função para redimensionar imagens
def resize_images(class_dir):
    for img_file in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_file)
        try:
            img = load_img(img_path, target_size=(224, 224))  # Redimensiona
            img.save(img_path)
        except Exception as e:
            print(f"Erro ao processar {img_path}: {e}")

for class_name in ["cat", "dog"]:
    resize_images(os.path.join(output_path, class_name))

# Carregar o modelo treinado
modelo = load_model("./models/trained_model.keras")

# Testar imagens e gerar matriz de confusão
y_true, y_pred = [], []
categories = ["cat", "dog"]

for class_index, class_name in enumerate(categories):
    for img_file in os.listdir(os.path.join(output_path, class_name)):
        img_path = os.path.join(output_path, class_name, img_file)

        img = load_img(img_path, target_size=(224, 224))
        img_array = np.expand_dims(img_to_array(img) / 255.0, axis=0)

        predicted_class = np.argmax(modelo.predict(img_array), axis=1)[0]
        y_true.append(class_index)
        y_pred.append(predicted_class)

# Criar Matriz de Confusão
conf_matrix = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", xticklabels=categories, yticklabels=categories)
plt.xlabel("Classe Predita")
plt.ylabel("Classe Real")
plt.title("Matriz de Confusão")
plt.show()

# Cálculo de Métricas
VP, FP = conf_matrix[0]
FN, VN = conf_matrix[1]

sensibilidade = VP / (VP + FN) if (VP + FN) else 0
acuracia = (VP + VN) / conf_matrix.sum() if conf_matrix.sum() else 0
precisao = VP / (VP + FP) if (VP + FP) else 0
f1_score = 2 * (precisao * sensibilidade) / (precisao + sensibilidade) if (precisao + sensibilidade) else 0

# Exibir resultados
print(f"Sensibilidade (Recall): {sensibilidade:.2f}")
print(f"Acurácia: {acuracia:.2f}")
print(f"Precisão: {precisao:.2f}")
print(f"F1-Score: {f1_score:.2f}")

# Gerar Curva ROC
fpr, tpr, _ = roc_curve(y_true, y_pred)

plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, marker='o', linestyle='-', color='b', label="Curva ROC")
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label="Linha Aleatória")
plt.xlabel("Taxa de Falsos Positivos (FPR)")
plt.ylabel("Taxa de Verdadeiros Positivos (TPR)")
plt.title("Curva ROC")
plt.legend()
plt.grid()
plt.show()


100%|██████████| 440M/440M [00:07<00:00, 65.1MB/s]

Extracting files...


Dataset baixado: /root/.cache/kagglehub/datasets/arifmia/animal/versions/1


ValueError: File not found: filepath=trained_model.keras. Please ensure the file is an accessible `.keras` zip file.